<a href="https://colab.research.google.com/github/boramkim0514/AI-Group-Project/blob/main/Global_COVID_19_Impact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

프레임워크 구축

In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

In [8]:
from google.colab import drive
drive.mount('/content/covid_19_clean_complete.csv')

Drive already mounted at /content/covid_19_clean_complete.csv; to attempt to forcibly remount, call drive.mount("/content/covid_19_clean_complete.csv", force_remount=True).


Load Data

In [9]:
df = pd.read_csv("/content/covid_19_clean_complete(1).csv")
df

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0,0,Europe
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0,0,Africa
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0,0,Europe
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0,0,Africa
...,...,...,...,...,...,...,...,...,...,...
49063,NaN,Sao Tome and Principe,0.186400,6.613100,2020-07-27,865,14,734,117,Africa
49064,NaN,Yemen,15.552727,48.516388,2020-07-27,1691,483,833,375,Eastern Mediterranean
49065,NaN,Comoros,-11.645500,43.333300,2020-07-27,354,7,328,19,Africa
49066,NaN,Tajikistan,38.861000,71.276100,2020-07-27,7235,60,6028,1147,Europe


0. 가설 세우기
* 인접 국가의 코로나 감염율 연관성 비교 
* 대륙 별/나라별 사망률 비교

#  1.집계 목적
카테고리  별 통계 현황 집계 **조회** 및 시각화
 * 날짜별 
 * 나라별 
 * 지역별 

In [10]:
df.tail()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
49063,NaN,Sao Tome and Principe,0.186400,6.613100,2020-07-27,865,14,734,117,Africa
49064,NaN,Yemen,15.552727,48.516388,2020-07-27,1691,483,833,375,Eastern Mediterranean
49065,NaN,Comoros,-11.645500,43.333300,2020-07-27,354,7,328,19,Africa
49066,NaN,Tajikistan,38.861000,71.276100,2020-07-27,7235,60,6028,1147,Europe
49067,NaN,Lesotho,-29.610000,28.233600,2020-07-27,505,12,128,365,Africa


#  2.데이터 살펴보기

## 2-1. NaN값 및 Data Type 확인

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49068 entries, 0 to 49067
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  14664 non-null  object 
 1   Country/Region  49068 non-null  object 
 2   Lat             49068 non-null  float64
 3   Long            49068 non-null  float64
 4   Date            49068 non-null  object 
 5   Confirmed       49068 non-null  int64  
 6   Deaths          49068 non-null  int64  
 7   Recovered       49068 non-null  int64  
 8   Active          49068 non-null  int64  
 9   WHO Region      49068 non-null  object 
dtypes: float64(2), int64(4), object(4)
memory usage: 3.7+ MB


In [12]:
df.describe()

,Lat,Long,Confirmed,Deaths,Recovered,Active
count,49068.000000,49068.000000,4.906800e+04,49068.000000,4.906800e+04,4.906800e+04
mean,21.433730,23.528236,1.688490e+04,884.179160,7.915713e+03,8.085012e+03
std,24.950320,70.442740,1.273002e+05,6313.584411,5.480092e+04,7.625890e+04
min,-51.796300,-135.000000,0.000000e+00,0.000000,0.000000e+00,-1.400000e+01
25%,7.873054,-15.310100,4.000000e+00,0.000000,0.000000e+00,0.000000e+00
50%,23.634500,21.745300,1.680000e+02,2.000000,2.900000e+01,2.600000e+01
75%,41.204380,80.771797,1.518250e+03,30.000000,6.660000e+02,6.060000e+02
max,71.706900,178.065000,4.290259e+06,148011.000000,1.846641e+06,2.816444e+06


## 2-2.  날짜 변환(datetime type)

In [13]:
df['Date'] = pd.to_datetime(df['Date'])
df['Date']

0       2020-01-22
1       2020-01-22
2       2020-01-22
3       2020-01-22
4       2020-01-22
           ...    
49063   2020-07-27
49064   2020-07-27
49065   2020-07-27
49066   2020-07-27
49067   2020-07-27
Name: Date, Length: 49068, dtype: datetime64[ns]

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49068 entries, 0 to 49067
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province/State  14664 non-null  object        
 1   Country/Region  49068 non-null  object        
 2   Lat             49068 non-null  float64       
 3   Long            49068 non-null  float64       
 4   Date            49068 non-null  datetime64[ns]
 5   Confirmed       49068 non-null  int64         
 6   Deaths          49068 non-null  int64         
 7   Recovered       49068 non-null  int64         
 8   Active          49068 non-null  int64         
 9   WHO Region      49068 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(4), object(3)
memory usage: 3.7+ MB


## 2-3. 날짜 데이터  분리(연도/월/**일**)

In [15]:
#df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49068 entries, 0 to 49067
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Country/Region  49068 non-null  object        
 1   Date            49068 non-null  datetime64[ns]
 2   Confirmed       49068 non-null  int64         
 3   Deaths          49068 non-null  int64         
 4   Recovered       49068 non-null  int64         
 5   Active          49068 non-null  int64         
 6   WHO Region      49068 non-null  object        
 7   Month           49068 non-null  int64         
 8   Day             49068 non-null  int64         
dtypes: datetime64[ns](1), int64(6), object(2)
memory usage: 3.4+ MB


# 3.데이터 전처리
 * NaN 값 처리(0, 평균값 채워주기)
 * 중복값(필요에 따라)


###  의미없는 불필요한 컬럼 삭제 (Province/State, Year, Long, Lat)

In [17]:
df.drop(['Province/State'], axis=1, inplace=True)

In [18]:
#df.drop(['Year'], axis=1, inplace=True)

In [19]:
df.drop(['Lat','Long'], axis=1, inplace=True)

In [20]:
#df.drop(['Date'], axis=1, inplace=True)

In [21]:
df_new = df.loc[(df['Confirmed'] == 0) & (df['Deaths'] == 0)].index
df_new_drop = df.drop(df_new, axis=0)
# df_idx = df.loc[df[feature_name] == 0].index
# df_new = df.drop(df_idx, axis=0)

In [22]:
#나라 이름 기준으로 compute ???
df_new_drop

,Country/Region,Date,Confirmed,Deaths,Recovered,Active,WHO Region,Month,Day
48,China,2020-01-22,1,0,0,1,Western Pacific,1,22
49,China,2020-01-22,14,0,0,14,Western Pacific,1,22
50,China,2020-01-22,6,0,0,6,Western Pacific,1,22
51,China,2020-01-22,1,0,0,1,Western Pacific,1,22
53,China,2020-01-22,26,0,0,26,Western Pacific,1,22
...,...,...,...,...,...,...,...,...,...
49063,Sao Tome and Principe,2020-07-27,865,14,734,117,Africa,7,27
49064,Yemen,2020-07-27,1691,483,833,375,Eastern Mediterranean,7,27
49065,Comoros,2020-07-27,354,7,328,19,Africa,7,27
49066,Tajikistan,2020-07-27,7235,60,6028,1147,Europe,7,27


In [27]:
plt.show

<function matplotlib.pyplot.show>

In [ ]:
df_new_drop.reset_index(drop=True, inplace=True) # index reset

In [ ]:
df_new_drop

In [ ]:
np.arange(1, 17).reshape(8, 2z)

In [ ]:
eg = np.arange(1, 73).reshape(2, 3, 4, 3)
eg[:,:,:,:]

In [ ]:
df_new.head()

In [ ]:
df.loc[df['Confirmed']==0]

In [ ]:
df.loc[df[['Confirmed' == 0] & ['Deaths']== 0]].idx

In [ ]:
df.loc[df['Deaths']==0]

###WHO 지역 구분 살펴보기

In [28]:
df['WHO Region'].unique()

array(['Eastern Mediterranean', 'Europe', 'Africa', 'Americas',
       'Western Pacific', 'South-East Asia'], dtype=object)

WHO 지역 기준으로 6개의 데이터셋으로 분류되어 있음을 알수 있다.



#  4.통계 현항 집계 조회 `groupby`
 * 날짜별 확진자/사망자 집계(총합계, 사망율)
 * 나라별 확진자/사망자 집계(총합계, 사망율)
 * 지역별 확진자/사망자 집계(총합계, 사망율)

###날짜  별 확진자/사망자 집계 조회

In [29]:
daily_stat = df.groupby("Date")['Confirmed','Deaths'].sum()#날짜별 확진자/사망자 총합계
daily_stat

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,Confirmed,Deaths
Date,,
2020-01-22,555,17
2020-01-23,654,18
2020-01-24,941,26
2020-01-25,1434,42
2020-01-26,2118,56
...,...,...
2020-07-23,15510481,633506
2020-07-24,15791645,639650
2020-07-25,16047190,644517


In [ ]:
before_covid = df2['2012-01-01':'2019-12-01']
after_covid = df2['2020-01':'2020-12']

before_covid = before_covid.groupby('월').mean()
after_covid_groupedby_year = after_covid.pivot('월','연도')

fig, axes = plt.subplots(2,1,
                         tight_layout=True
                         )

axes[0].plot(before_covid['구인인원'], label='코로나 이전(2012-2019)')
axes[0].plot(after_covid_groupedby_year['구인인원'], label='코로나 이후(2020)')
axes[0].legend(prop={'size': 18})
axes[1].fill_between(df_new_covid_time2.index, df_new_covid_time2['new'])
plt.title('2020년 코로나 신규확진자 추이', fontsize=30)
plt.show()

### 국가별 확진자/사망자 집계 조회

In [34]:
country_stat = df.groupby("Country/Region")['Confirmed','Deaths'].sum()
country_stat

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,Confirmed,Deaths
Country/Region,,
Afghanistan,1936390,49098
Albania,196702,5708
Algeria,1179755,77972
Andorra,94404,5423
Angola,22662,1078
...,...,...
West Bank and Gaza,233461,1370
Western Sahara,901,63
Yemen,67180,17707


###  지역별 확진자/사망자 집계 조회

In [35]:
regional_stat= df.groupby("WHO Region")['Confirmed','Deaths'].sum()
regional_stat

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,Confirmed,Deaths
WHO Region,,
Africa,21791827,439978
Americas,402261194,19359292
Eastern Mediterranean,74082892,1924029
Europe,248879793,19271040
South-East Asia,55118365,1458134
Western Pacific,26374411,932430


In [ ]:
x = np.arange(0, 10, 0.1)
y_1 = 
y_2 = 1 + np.cos(x)

plt.plot(x, y_1, color='blue', alpha=0.3, label="1+sin")
plt.plot(x, y_2, color='red', alpha=0.7, label="1+cos")

plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(60, 40))
x_label = ["Africa", "Americas", "Eastern Mediterranean", "Europe", "South-East Asia", "Western Pacific"]
x = np.arange(len(x_label)) # WHO Reigion을 label로 배열 생성

y_1 = [21791827, 402261194, 74082892, 248879793, 55118365, 26374411] # 확진자
y_2 = [439978, 19359292, 1924029, 19271040, 1458134, 932430] # 사망자

fig, axes = plt.subplots() # 1, 1 형태의 ax 하나 생성.

# bar의 넓이 넓이 지정
width = 0.35
axes.bar(x - width/2, y_1, width, align='center', alpha=0.5)
axes.bar(x + width/2, y_2, width, align='center', alpha=0.8)

# xticks 설정
plt.xticks(x)

axes.set_xticklabels(x_label) # 실제 표시할 레이블
plt.legend()
plt.show()

In [ ]:
x = np.arange(1, 10, 0.05) # np.linspace
x_label = ["Africa", "Americas", "Eastern Mediterranean", "Europe", "South-East Asia", "Western Pacific"]
x = np.arange(len(x_label)) # WHO Reigion을 label로 배열 생성

y_1 = [21791827, 402261194, 74082892, 248879793, 55118365, 26374411] # 확진자
y_2 = [439978, 19359292, 1924029, 19271040, 1458134, 932430] # 사망자

# y_1 = np.cos(x) + 1
# y_2 = np.sin(x) + 1
y_3 = y_1 * y_2 / np.pi

plt.fill_between(x, y_1, color='green', alpha=0.1)
plt.fill_between(x, y_2, color='blue', alpha=0.2)
plt.fill_between(x, y_3, color='red', alpha=0.3)

plt.show()

In [ ]:
plt.show('Confirmed')

In [ ]:
sns.violinplot(x=df_Africa['Confirmed'])

In [ ]:
sns.catplot(
    x='Confirmed',
    y='Deaths',
    data=regional_stat,
    kind='bar')
plt.show()

In [ ]:
sns.displot(df['Confirmed'])
plt.show()

In [ ]:
sns.displot(regional_stat)
plt.show()

In [ ]:
# bar 
sns.countplot(
    x='Country/Region',  
    y='Confirmed',   
    kind='bar'
)

plt.show()

In [ ]:
sns.countplot(x='WHO Region', data=regional_stat, hue='WHO Region')

In [ ]:
plt.plot(df['Confirmed'])
plt.plot(df['Deaths'])
plt.plot(df['Active'])
plt.plot(df['Recovered'])
plt.grid()
plt.legend(['Confirmed','Deaths','Active','Recovered'])
palette='coolwarm'
plt.show()

#  5.피벗테이블 사용
    * 국가별 코로나 확진자의 사망자율/회복률을 지역별로 조회하기 위해 
    * 행 인덱스: 국가
    * 열 인덱스: WHO 지역
    * 집계할값 대상: 사망자 회복자/사망자

###  지역별 코로나 확진자 합계 조회

In [47]:
pd.pivot_table(
    df, #사용할 데이터 프레임
    index=["Date"],  #나라별로 보고 싶어서 선택한 인덱스 #날짜
    columns="WHO Region",  #규모구분의 값인 WHO Region 컬럼명이 됨 #확진자
    values="Confirmed",
    aggfunc=np.sum
    ) #확진자 수   #사망자

WHO Region,Africa,Americas,Eastern Mediterranean,Europe,South-East Asia,Western Pacific
Date,,,,,,
2020-01-22,0,1,0,0,2,552
2020-01-23,0,1,0,0,3,650
2020-01-24,0,2,0,2,5,932
2020-01-25,0,2,0,3,8,1421
2020-01-26,0,6,0,3,9,2100
...,...,...,...,...,...,...
2020-07-23,656696,8294228,1439937,3216701,1625727,277192
2020-07-24,677376,8460627,1453830,3239712,1679154,280946
2020-07-25,694057,8609554,1467209,3259047,1732350,284973


### 지역별 코로나 사망자 합계 조회

In [ ]:
pivot = df.pivot_table(
    index='Date', 
    columns="WHO Region", 
    values='Deaths')
pivot

###  지역별 코로나 사망자 월별 조회

In [ ]:
# pivot = df.pivot_table(
#     index='Country/Region', 
#     columns="WHO Region", 
#     values='Deaths')
# pivot.head(20)


In [42]:
pd.pivot_table(
    df, #사용할 데이터 프레임
    index=["Month"],  #나라별로 보고 싶어서 선택한 인덱스 #날짜
    columns="WHO Region",  #규모구분의 값인 WHO Region 컬럼명이 됨  #확진자
    values="Deaths") #확진자 수   #사망자

WHO Region,Africa,Americas,Eastern Mediterranean,Europe,South-East Asia,Western Pacific
Month,,,,,,
1,0.000000,0.000000,0.000000,0.000000,0.000000,1.616364
2,0.000000,0.000750,0.268025,0.052155,0.000000,29.205016
3,0.430108,20.490182,52.435484,91.712903,4.332258,60.521994
4,11.761806,876.933333,251.407576,1141.395417,107.080000,88.913333
5,36.018817,2679.362553,452.259531,2046.935887,461.354839,121.627566
6,89.975000,4553.492754,828.981818,2372.012083,1462.210000,131.544848
7,184.599537,6453.553140,1458.920875,2562.450000,3122.162963,141.843771


컬럼과 인덱스를 바꿔서 보니 더 보기 편하다

In [ ]:
import numpy as np
pd.pivot_table(
    df, #사용할 데이터 프레임
    index="WHO Region", 
    columns="Country/Region", 
    values="Deaths")  #  사망자수

In [ ]:
df.groupby(["WHO Region","Country/Region"])["Confirmed","Deaths"].sum()

In [ ]:
sns.barplot(x='WHO Region', y='Confirmed', data=df, palette='coolwarm')

In [ ]:
sns.barplot(x='WHO Region', y='Deaths', data=df, palette='coolwarm')

In [ ]:
# Survival_Rate = Regions_Deaths/Region_Confirmed
# Survival_Rate

In [ ]:
import numpy as np
pd.pivot_table(
    df, #사용할 데이터 프레임
    index="Country/Region",
    columns="WHO Region",
    values="Deaths")

#  6.시각화

Heatmap을 이용한 상관관계 조사

In [ ]:
df['WHO Region'].unique()

In [ ]:
plt.rcParams["figure.figsize"] =(8, 5)
sns.heatmap(df.corr(), annot=True, linewidths=1)
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] =(8, 5)
sns.heatmap(df.corr(), annot=True, linewidths=1, cmap="Reds")
plt.show()

In [ ]:
df_Europe = df.loc[df['WHO Region'] == 'Europe']
df_Europe

In [ ]:
df_Americas = df.loc[df['WHO Region'] == 'Americas']
df_Americas

In [ ]:
df_Africa = df.loc[df['WHO Region'] == 'Africa']
df_Africa.head()

In [ ]:
sns.heatmap(df_Europe.corr(), cmap ='Reds', annot=True)
plt.show()

In [ ]:
sns.heatmap(df_Africa.corr(), cmap ='Reds', annot=True)
plt.show()

In [ ]:
sns.heatmap(df_Americas.corr(), cmap ='Reds', annot=True)
plt.show()

In [ ]:
sns.jointplot(x='onfirmed', y='Deaths', height=5, data=df, kind='kde', color='g')

In [ ]:
sns.pairplot(df_Africa)
plt.show()

In [ ]:
sns.countplot(x='WHO Region', data=df, palette="coolwarm")
plt.show()

In [ ]:
sns.barplot(x='Month', y='Confirmed', data=df, palette="coolwarm")
plt.show()

In [ ]:
sns.barplot(x='Day', y='Confirmed', data=df, palette="coolwarm")
plt.show()

In [ ]:
sns.countplot(df['Month'], palette="coolwarm")
plt.show()

In [ ]:
sns.pairplot(df_Africa)

In [ ]:
sns.countplot(
    x= 'Country/Region',
    data= df,
    hue='WHO Region'
)

plt.show()

In [ ]:
sns.countplot(
    x= 'WHO Region',
    data= df,
    hue='WHO Region'
)

plt.show()

In [ ]:
sns.countplot(x='WHO Rregion', data=df)
plt.show()

In [ ]:
Region_Confirmed.plot(kind='barh')

In [ ]:
df_Africa = df.loc[df['WHO Region','Confirmed'] == ['Africa']
df_Africa

# plt.rcParams["figure.figsize"] = (12, 30)
# df_Africa.plot(kind='barh')

In [ ]:
df_Africa = df.loc[df['WHO Region','Confirmed'] == ['Africa']
df_Africa.head(100)

# 가설 확인 
지역별 생존률 컬럼을 어떻게 추가하나?

In [ ]:
sample_df = pd.pivot_table(
    df, #사용할 데이터 프레임
    index="WHO Region",  #나라별로 보고 싶어서 선택한 인덱스 #날짜
    values=["Confirmed", "Deaths"],
    aggfunc=np.max
    ) #확진자 수   #사망자

sample_df.head()

In [ ]:
sample_df = sample_df.reset_index()
sample_df.head()

In [ ]:
sns.barplot(x="WHO Region", y=[['Confirmed', "Deaths"]], data=sample_df)

plt.show()

In [ ]:
sample_df.plot(x="WHO Region", y=["Confirmed, Deaths"], kind='bar')